In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image

def load_image(image_path):
    img = Image.open(image_path)
    return np.array(img)

def equirectangular_to_cartesian(u, v, width, height):
    theta = 2 * np.pi * u / width
    phi = np.pi * v / height - np.pi / 2
    x = np.cos(phi) * np.cos(theta)
    y = np.cos(phi) * np.sin(theta)
    z = np.sin(phi)
    return x, y, z

def map_to_sphere(image, sphere_radius):
    height, width, _ = image.shape
    sphere_image = np.zeros_like(image)

    for v in range(height):
        for u in range(width):
            x, y, z = equirectangular_to_cartesian(u, v, width, height)
            theta = np.arctan2(y, x)
            phi = np.arctan2(z, np.sqrt(x**2 + y**2))

            # Convert theta and phi to pixel coordinates
            pixel_u = int((theta / (2 * np.pi) + 0.5) * (width - 1))
            pixel_v = int((phi / np.pi + 0.5) * (height - 1))

            sphere_image[v, u] = image[pixel_v, pixel_u]

    return sphere_image

if __name__ == "__main__":
    # Replace 'sample_image.jpg' with the path to your sample image.
    input_image_path = "/Users/akotamraju/Documents/warriors-news-photo-3.png"

    # Load the image
    image = load_image(input_image_path)

    # Radius of the sphere (you can adjust this value based on your preference)
    sphere_radius = 200

    # Map the image onto a sphere
    sphere_image = map_to_sphere(image, sphere_radius)

    # Create a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    height, width, _ = sphere_image.shape
    for v in range(height):
        for u in range(width):
            x, y, z = equirectangular_to_cartesian(u, v, width, height)
            color = sphere_image[v, u] / 255.0  # Normalize pixel values to [0, 1]

            color = color.reshape(1,-1)

            ax.scatter(x * sphere_radius, y * sphere_radius, z * sphere_radius, c=color, marker='o', s=1)

    ax.set_title("Spherical Image")
    ax.set_axis_off()
    plt.show()
